In [1]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

# temporary solution for relative imports in case sadl is not installed
# if sadl is installed, no need to use the line
import sys
import os
import inspect
from inspect import signature
import numpy as np
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

from SADL.static_data.algorithms import pyod

kwargs = {"algorithm_": "cblof","contamination":2, "label_parser": True}
model = pyod.PyodAnomalyDetection(**kwargs)

#print(model.algorithm_)
print(model.label_parser)
print(model.get_params())
#print(signature(model.__class__).parameters.values())

kwargs = {"algorithm_": "abod","contamination":2, "label_parser": False}
model.set_params(**kwargs)
print(model.label_parser)
print(model.get_params())


2024-02-02 11:11:50.467401: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 11:11:50.467452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 11:11:50.468793: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-02 11:11:50.476047: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-02 11:11:51.598560: W tensorflow/compiler/tf2

True
{'label_parser': True, 'algorithm_': 'CBLOF', 'alpha': 0.9, 'beta': 5, 'check_estimator': False, 'clustering_estimator': None, 'contamination': 2, 'n_clusters': 8, 'random_state': None, 'use_weights': False}
False
{'label_parser': False, 'algorithm_': 'ABOD', 'contamination': 2, 'method': 'fast', 'n_neighbors': 5}


Data generation

In [2]:
# Define the number of inliers and outliers
n_samples = 200
outliers_fraction = 0.25

# Compare given detectors under given settings
# Initialize the data
xx, yy = np.meshgrid(np.linspace(-7, 7, 100), np.linspace(-7, 7, 100))
n_inliers = int((1. - outliers_fraction) * n_samples)
n_outliers = int(outliers_fraction * n_samples)
ground_truth = np.zeros(n_samples, dtype=int)
ground_truth[-n_outliers:] = 1

# Show the statics of the data
print('Number of inliers: %i' % n_inliers)
print('Number of outliers: %i' % n_outliers)
print('Ground truth shape is {shape}. Outlier are 1 and inliers are 0.\n'.format(shape=ground_truth.shape))
print(ground_truth)

X1 = 0.3 * np.random.randn(n_inliers // 2, 2)
X2 = 0.3 * np.random.randn(n_inliers // 2, 2)
X = np.r_[X1, X2]
# Add outliers
X = np.r_[X, np.random.uniform(low=-6, high=6, size=(n_outliers, 2))]


Number of inliers: 150
Number of outliers: 50
Ground truth shape is (200,). Outlier are 1 and inliers are 0.

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


Model Fitting - errors


In [3]:
%load_ext autoreload
%autoreload 2
kwargs = {"algorithm_": "abod","contamination":2, "label_parser": True, "n_neighbors": 4, "method": 'method'}
model = pyod.PyodAnomalyDetection(**kwargs)

# Fit the model using SADL Library - error en fit
model.fit(X)
kwargs = {"algorithm_": "abod","contamination":0.1, "label_parser": True, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)
#print(model.model)
# Fit the model using SADL Library - sin error
#model.fit(X)
scores_pred = model.decision_function(X) * -1
print(scores_pred)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
PYODerror fit(): ('method', 'is not a valid method')
For further reference please see: https://pyod.readthedocs.io/en/latest/
PYODerror decision_function(): This ABOD instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.
For further reference please see: https://pyod.readthedocs.io/en/latest/


TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

Model Fitting - OK


In [4]:
%load_ext autoreload
%autoreload 2
kwargs = {"algorithm_": "abod","contamination":0.1, "label_parser": True, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)

print(model.model)
# Fit the model using SADL Library - sin error
model.fit(X)
scores_pred = model.decision_function(X) * -1
print(scores_pred)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
ABOD(contamination=0.1, method='fast', n_neighbors=4)
[2.00742680e+04 2.61594407e+05 2.89671840e+05 6.23510799e+03
 2.61370936e+04 5.76874981e+03 2.67231392e+04 8.69747908e+04
 2.70990105e+04 1.77489829e+05 2.75492765e+04 2.48711875e+03
 2.17996633e+05 3.64998012e+05 6.13654704e-01 9.62244215e+04
 3.27962225e+05 7.02786216e+03 2.37963588e+04 1.17034144e+03
 4.23112769e+04 6.91580424e+04 1.77605312e+04 2.32009189e+03
 2.23746203e+06 2.65300165e+05 3.40272510e+03 1.73024313e+04
 1.69887581e+05 5.34499341e+04 5.72941416e+05 3.50213254e+03
 1.86574188e+04 7.16738500e+04 1.37686533e+03 4.46463977e+04
 2.14776136e+03 1.01891942e+05 3.97898464e+05 4.51927940e+03
 5.57114909e+03 6.02617149e+03 3.69113492e+04 1.50895901e+03
 4.98515042e+03 1.08882273e+04 1.43109287e+05 7.63449689e+02
 5.52060976e+03 4.70369798e+03 5.88841256e+01 2.82930005e+04
 8.40788431e+03 3.81664048e+00 1.79305553e+04 2.38994402e+04
 2.1

Model Predict w/ label_parser function

In [5]:
%load_ext autoreload
%autoreload 2

def function(X):
    print("Estoy en funcion")
    return X < 0.5

kwargs = {"algorithm_": "abod","contamination":0.1, "label_parser": function, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)

print(model.get_params())
model.predict(X)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'label_parser': <function function at 0x7f08f7126d40>, 'algorithm_': 'ABOD', 'contamination': 0.1, 'method': 'fast', 'n_neighbors': 4}
Estoy en funcion


array([[ True, False],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True, False],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ Tr

Model Predict wo/ label_parser

In [6]:
%load_ext autoreload
%autoreload 2


def function(X):
    print("Estoy en funcion")
    return X < 0.5

kwargs = {"algorithm_": "abod","contamination":0.1, "label_parser": None, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)

print(model.get_params().keys())
model.fit(X)
model.predict(X)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
dict_keys(['label_parser', 'algorithm_', 'contamination', 'method', 'n_neighbors'])


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 0])

In [29]:
%load_ext autoreload
%autoreload 2

import sys
import os
from pyod.models.lscp import LSCP
from pyod.models.lof import LOF
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
from SADL.static_data.algorithms.pyod import PyodAnomalyDetection

kwargs_lof = {"algorithm_": "lof"}

detector_list = [pyod.PyodAnomalyDetection(**kwargs_lof).model, pyod.PyodAnomalyDetection(**kwargs_lof).model]

kwargs = {"algorithm_": "lscp", "detector_list":detector_list,"random_state": 42}
model = pyod.PyodAnomalyDetection(**kwargs)
print(model.get_params())


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'label_parser': None, 'algorithm_': 'LSCP', 'contamination': 0.1, 'detector_list': [LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, novelty=True, p=2), LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, novelty=True, p=2)], 'local_max_features': 1.0, 'local_region_size': 30, 'n_bins': 10, 'random_state': 42}
